In [1]:
import re
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))
n = 4

In [2]:
def parse(line):
    user, follower = line.strip().split('\t')
    return (int(follower), int(user))

In [3]:
rels = sc.textFile('/data/twitter/twitter_sample_small.txt', 16).map(parse).cache()

In [4]:
source = 12
target = 34

In [5]:
vertex = source
path = []
distances = sc.parallelize([(vertex, None)]).partitionBy(n)
completed = sc.parallelize([(vertex, [])]).partitionBy(n)

In [6]:
def map_completed(vertex, item, path):
    p = list(path)
    p.append(item)
    
    return (vertex, p)

In [7]:
while True:
    d_join = distances.join(rels, n)
    
    c = d_join\
        .join(completed, n)\
        .map(lambda (v, (i, p)): map_completed(i[1], v, p))\
        .partitionBy(n)

    distances = d_join.map(lambda (v, i): (i[1], v))
    completed = completed.union(c).reduceByKey(lambda x, y: x if len(x) <= len(y) else y)
    
    completed_target = completed.filter(lambda (v, p): v == target)
    
    if completed_target.count():
        path = completed_target.first()[1]
        path.append(target)
        break

In [8]:
print(','.join(map(str, path)))

12,422,53,52,107,20,23,274,34
